In [90]:
import requests
import time
import hashlib
import pandas as pd
import json


Definindo

In [131]:
PUBLIC_KEY = "0b08b7e93738e4973b80ae94c2986e7d"
PRIVATE_KEY = "292deed8848488aa3574b5eae82f97b0f22da73c"

ts = int(time.time())

hash_md5 = hashlib.md5(f"{ts}{PRIVATE_KEY}{PUBLIC_KEY}".encode()).hexdigest()

url = "http://gateway.marvel.com/v1/public/characters"

params = {
    "ts": ts,
    "apikey": PUBLIC_KEY,
    "hash": hash_md5
    }

response = requests.get(url, params=params)

In [132]:
data = response.json()
characters_list = data['data']['results']

df = pd.json_normalize(characters_list)


In [133]:
selected_columns = [
        "id",
        "name",
        #"description",
        "modified",
        "comics.available",
        "comics.returned",
        "series.available",
        "series.returned",
        "stories.available",
        "stories.returned",
        "events.available",
        "events.returned"
    ]

df = df[selected_columns]

print(df.head(90))

         id                        name                  modified  \
0   1011334                     3-D Man  2014-04-29T14:18:17-0400   
1   1017100                A-Bomb (HAS)  2013-09-18T15:54:04-0400   
2   1009144                      A.I.M.  2013-10-17T14:41:30-0400   
3   1010699                 Aaron Stack  1969-12-31T19:00:00-0500   
4   1009146  Abomination (Emil Blonsky)  2014-06-27T19:39:07-0400   
5   1016823      Abomination (Ultimate)  2012-07-10T19:11:52-0400   
6   1009148               Absorbing Man  2016-05-25T12:04:23-0400   
7   1009149                       Abyss  2014-04-29T14:10:43-0400   
8   1010903   Abyss (Age of Apocalypse)  1969-12-31T19:00:00-0500   
9   1011266                Adam Destine  1969-12-31T19:00:00-0500   
10  1010354                Adam Warlock  2016-05-05T15:34:56-0400   
11  1010846        Aegis (Trey Rollins)  1969-12-31T19:00:00-0500   
12  1017851                 Aero (Aero)  2021-08-27T17:52:34-0400   
13  1012717             Agatha Har